# Garbled

Goal: To create a program that can read in a text input, and return nonsense sentences that sound similar to the fed text

## pronouncingpy + spaCy

Using pronouncingpy and spaCy to get a list of all the phonemes in a text

In [1]:
import pronouncing as pr
import spacy

In [2]:
nlp = spacy.load("en_core_web_md")

In [48]:
def get_all_phones(file_path):
    file = open(file_path).readlines()
    all_phones = []
    
    for line in file:
        line = nlp(line)
        
        for word in line:
            phones = pr.phones_for_word(word.text.lower())

            if len(phones) > 0:
                all_phones.append(phones[0])
        
        all_phones.append("/n")
        
    return all_phones

## Markovify Phonemes

Using markovify to predict phonemes in a text

In [27]:
import markovify

In [66]:
def markovify_phones(phones_text):
    markovified_text = markovify.Text(phones_text, 2)
    sentence = markovified_text.make_short_sentence(75, tries=100)
    return sentence

`create_words_from_phones` Tasks:

1. Get first phone in list
2. Search for matching word(s)
3. If not found, append next phone to end
4. Repeat 2-3 until word(s) found
5. If word not found, take original phone and pull a list of words that contains that phone sound
6. Once words documented, move to next phone that hasn't been used yet, repeat 2-3

In [72]:
def create_words_from_phones(sen):
    sen_phones = sen.split()
    search = [sen_phones[0]]
    LENGTH = len(sen_phones) - 1
    
    cur = 0
    last = 0
    
    all_results = []

    while cur <= LENGTH:
        result_found = False

        while result_found == False:
            search_str = ""

            for phone in search:
                search_str += phone + " "

            results = pr.search("^" + search_str.rstrip() + "$")

            cur += 1

            if len(results) == 0:
                if cur <= LENGTH:
                    search.append(sen_phones[cur])
                    
                else:
                    word_with_phone = pr.search(sen_phones[last])
                    all_results.append(word_with_phone)

                    last += 1
                    cur = last

                    search = [sen_phones[cur]]
                    
            else:
                all_results.append(results)
                result_found = True
                
                if cur <= LENGTH:
                    last = cur
                    search = [sen_phones[last]]


    return all_results

### Put it All Together

In [73]:
import random

In [74]:
text_phones = get_all_phones("txt_examples/genesis.txt")
text_phones = " ".join(text_phones)

In [79]:
phone_sentence = markovify_phones(text_phones)
phone_sentence = phone_sentence.replace("/n","")
results = create_words_from_phones(phone_sentence)

print(phone_sentence)
print("")

sentence = ""
for result in results:
    sentence += random.choice(result) + " "
    
print(sentence)

IH0 N DH AH0 ER1 TH  S OW1  AH0 N T L IY0 DH AH0 TH ER1 D D EY1 

in the ur ths ow a kinne stiller's exploratory stoically the third dey 
